#### Main  notebook for the Kaggle google - Capstone

In [ ]:
%pip install google-adk==1.18.0

### Install google adk via pip and import the necessary class/utils/functions

In [21]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent,LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Setup the google API Keys and model to be used and Re-try configs

In [ ]:
### setup the configs and keys
import os
from dotenv import load_dotenv

load_dotenv()
if "GOOGLE_API_KEY" not in os.environ:
    raise ValueError("GOOGLE_API_KEY not found in environment variables.")
    
google_llm_model_choice = "gemini-2.5-flash"

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

### Build the tool to get External Market Mood Data from Ticker Tape Tool .  
Use HTTP Call to get the 'market mood data' and extract for the current day only

In [23]:
import httpx 
import json
import math
market_mood_url = "https://api.tickertape.in/mmi/now"


def fetch_market_mood():

    data= httpx.get(
        url=market_mood_url,
        headers={"Accept": "application/json"})
    
    market_mood_data={}

    if(data.status_code ==200):
        raw_json = json.loads(data.content)
        if raw_json and raw_json['success']  :
            market_mood_data = {'nifty':raw_json['data']['nifty'],  'mood_index':math.floor(raw_json['data']['indicator'])}
                            
    
    return market_mood_data

data = fetch_market_mood()
print(data)


market_mood_agent = Agent(
    name="MarketMoodAgent",
    model=Gemini(
        model=google_llm_model_choice,
        retry_options=retry_config
    ),
    instruction="""The market mood agent get the latest market data for today. The data includes nifty value and mood index.
    If mood_index is less than 30 market mood is extreme fear, if mood_index is between 30 and 50 market mood is fear,
      if mood_index is between 50 and 70 market mood is greed, if mood_index is above 70 market mood is extreme greed.""",
    tools=[fetch_market_mood],
    output_key="market_mood",  # The result will be stored with this key.
)

print("✅ market_mood created.")

{'nifty': 26175.75, 'mood_index': 59}
✅ market_mood created.


## Local callback server
This notebook starts a small HTTP server on `127.0.0.1:<port>` to receive the redirect after Zerodha login.
1. Start the server cell (choose a free `port`, e.g. `8080`).
2. Call `login(redirect_uri='http://127.0.0.1:8080')` from the `zerodha_profolio_tool` module (it will open the login URL).
3. After completing login in the browser, Zerodha will redirect to the local server and the notebook will capture `request_token`.

In [24]:
# Implement a tiny callback HTTP server to capture the `request_token`
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading
from urllib.parse import urlparse, parse_qs
import time
from zerodha_profolio_tool import login,generate_session,get_current_holdings

_token_data = {}

class TokenHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        parsed = urlparse(self.path)
        qs = parse_qs(parsed.query)
        token = qs.get('request_token') or qs.get('requestToken')
        if token:
            _token_data['request_token'] = token[0]
            message = '<html><body><h2>Login successful. You can close this tab.</h2></body></html>'
        else:
            message = '<html><body><h2>No request_token found in URL.</h2></body></html>'
        self.send_response(200)
        self.send_header('Content-type','text/html')
        self.end_headers()
        self.wfile.write(message.encode('utf-8'))

def start_server(port: int = 8080):
    server = HTTPServer(('127.0.0.1', port), TokenHandler)
    thread = threading.Thread(target=server.serve_forever, daemon=True)
    thread.start()
    return server

def get_request_token(timeout: int = 300):
    start = time.time()
    while time.time() - start < timeout:
        if 'request_token' in _token_data:
            return _token_data.pop('request_token')
        time.sleep(0.5)
    raise TimeoutError('No request_token received within timeout')

def stop_server(server: HTTPServer):
    try:
        server.shutdown()
        server.server_close()
    except Exception as e:
        print('Error stopping server:', e)

In [25]:
# Example usage: start the server, call login(), then wait for token.
# 1) Start server on a free port (use same port in redirect URI).
server = start_server(port=8080)
print('Server started at http://127.0.0.1:8080')

# 2) Trigger Zerodha login (this opens the browser). Make sure your redirect URI is set to http://127.0.0.1:8080
# from zerodha_profolio_tool import login
login_url, kite = login(open_in_browser=True)
print('Please complete login at:', login_url)


Server started at http://127.0.0.1:8080
Please complete login at: https://kite.zerodha.com/connect/login?api_key=nqmwjvyipiesbf0h&v=3


### Setup Authentication and Token generation for Zerodha Stock broker .
 - You will need to generate an 'request+token' using 2FA - Authenticator and use it with API Key/Secret to start a session.
 - Session is valid for 12 hours, so once obtained it can be reused across  calls.
 - Session will be used by the LLM tool to extract the current stock holdings. 
 - A new tool 'PortfolioExpert' will use the method 'get_current_portfolio' method to obtain a strucutured JSON containing current Stocks


In [29]:
print(_token_data)
kite,access= generate_session(kite, request_token=_token_data.get('request_token'))
# Uncomment and run the above two lines after you have `KITE_API_KEY` and `KITE_API_SECRET` set in environment or .env.

# 3) Wait for request token (this will block until token is received or timeout)
# request_token = get_request_token(timeout=300)
# print('Received request_token:', request_token)

# 4) Exchange request_token for access token (example):
# data = kite.generate_session(request_token, os.getenv('KITE_API_SECRET'))
# access_token = data['access_token']
# kite.set_access_token(access_token)
# print('Access token:', access_token)


get_current_holdings(kite)

{'request_token': '28Jer46UsFJb6k1achpJmYdVOMrAjfSH'}


[{'tradingsymbol': 'ARE&M',
  'isin': 'INE885A01032',
  'price': 0,
  'quantity': 10,
  'average_price': 545,
  'last_price': 950.45,
  'close_price': 951.9},
 {'tradingsymbol': 'CEATLTD',
  'isin': 'INE482A01020',
  'price': 0,
  'quantity': 1,
  'average_price': 1322.8,
  'last_price': 3839.5,
  'close_price': 3849.9},
 {'tradingsymbol': 'CESC',
  'isin': 'INE486A01021',
  'price': 0,
  'quantity': 30,
  'average_price': 180,
  'last_price': 172.05,
  'close_price': 170.65},
 {'tradingsymbol': 'CHAMBLFERT',
  'isin': 'INE085A01013',
  'price': 0,
  'quantity': 6,
  'average_price': 302.7,
  'last_price': 441.7,
  'close_price': 440.15},
 {'tradingsymbol': 'ENDURANCE',
  'isin': 'INE913H01037',
  'price': 0,
  'quantity': 5,
  'average_price': 1148.5,
  'last_price': 2689.6,
  'close_price': 2643.6},
 {'tradingsymbol': 'ESCORTS',
  'isin': 'INE042A01014',
  'price': 0,
  'quantity': 1,
  'average_price': 1199.9,
  'last_price': 3844.2,
  'close_price': 3817.4},
 {'tradingsymbol': 'GAI

In [36]:
def get_current_portfolio()-> list[dict[str, any]]:

    """
    Fetches and returns the current portfolio holdings from Zerodha Kite. 
    Returns:
        list[dict(str, any)]: A list of dictionaries representing the current holdings.
          Example list item : {'tradingsymbol': 'REDINGTON','quantity': 12, 'average_price': 160.966667, 'last_price': 284.85,'close_price': 281.2}
    """
    return get_current_holdings(kite)

In [47]:
portfolio_expert_agent = LlmAgent(
    name="PortfolioExpert",
    model=Gemini(
        model="gemini-2.5-flash",
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a stock protfolio analyst . Your goal is to You may calculate portfolio-level metrics (total value, cash, exposure, allocation by symbol/sector/country, % weights).
    Use the get_current_portfolio tool to get my current stock holdings. Give me sectorwise allocation of stocks and also total profit loss .""",
    tools=[get_current_portfolio],
    output_key="portfolio_data",
)

print("✅ portfolio agent created.")

✅ portfolio agent created.


In [ ]:
### visual generator agent 

market_visual_agent = LlmAgent(
    name="MarketVisualAgent",
    model=Gemini(
        model=google_llm_model_choice,
        retry_options=retry_config
    ),
    instruction="""The Market Visual  agent get current market mood from {market_mood}. Generate a pie chart with the mood index distribution.
    If mood_index is less than 30 market mood is extreme fear, if mood_index is between 30 and 50 market mood is fear,
      if mood_index is between 50 and 70 market mood is greed, if mood_index is above 70 market mood is extreme greed.
    Plot the current as pointer on pie chart""",
    output_key="market_visual",  # The result will be stored with this key.
)

print("✅ MarketVisualAgent created.")

In [ ]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model=google_llm_model_choice,
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine the market mood data and portfolio summary data into a single executive summary:

    **Market Mood Data**
    {market_mood}
    
    **Potfolio Summary Data:**
    {portfolio_data}
    
     
    
    The final summary should be around 400 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

In [ ]:
# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.


# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[market_mood_agent, portfolio_expert_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="EndtoEndPortfolioAnalys",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Provide a detailed analysis on my stock portfolio give me current profit loss and sectorwise distribution", verbose=True
)


 ### Created new session: debug_session_id

User > Provide a detailed analysis on my stock portfolio give me current profit loss and sectorwise distribution
PortfolioExpert > [Calling tool: get_current_portfolio({})]
PortfolioExpert > [Tool result: {'result': [{'tradingsymbol': 'ARE&M', 'isin': 'INE885A01032', 'price': 0, 'quantity': 10, 'average_...]
PortfolioExpert > Here's a detailed analysis of your stock portfolio:

**Current Profit/Loss:**

I've calculated your profit or loss for each holding based on the `last_price` and your `average_price`:

*   **ARE&M:** (950.45 - 545) * 10 = ₹4054.5
*   **CEATLTD:** (3839.5 - 1322.8) * 1 = ₹2516.7
*   **CESC:** (172.05 - 180) * 30 = -₹238.5
*   **CHAMBLFERT:** (441.7 - 302.7) * 6 = ₹834.0
*   **ENDURANCE:** (2689.6 - 1148.5) * 5 = ₹7705.5
*   **ESCORTS:** (3844.2 - 1199.9) * 1 = ₹2644.3
*   **GAIL:** (175.45 - 93.633333) * 45 = ₹3681.75
*   **GRANULES:** (568.7 - 321.05) * 5 = ₹1238.25
*   **HEIDELBERG:** (181.14 - 262) * 7 = -₹566.02
*   